In [1]:
# transformers is not preinstalled in google colab
# !pip install transformers

# weights and biases is not preinstalled in google colab
# !pip install wandb -q

# jupyter setup
# % commands are "jupyter notbook commands" - %matplotlib inline allows for inline plotting
%matplotlib inline
 
# import modules
import os 
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn import model_selection
import torch
import torch.nn as nn
import transformers 
# import wandb
import matplotlib.pyplot as plt
import time
import random
from pathlib import Path
from typing import List, Optional, Tuple, Union

In [2]:
# set parameters
parameter_dict = {
    'huggingface_model': 'bert-base-uncased', 
    'epochs' : 4,
    'batch_size': 10,
    'dropout_finetune': 0.2,
    'learning_rate_AdamW': 2e-5,
    'metadata_embedding_size': 128,
    'hidden_layer_size': 2048,
    'seed': 101
} 

In [3]:
model_name = f"{parameter_dict['huggingface_model']}-with-metadata-2hidden_layers"

In [4]:
# Set up torch for colab
if torch.cuda.is_available():
  device = torch.device('cuda') # specify the GPU we want to use in torch
  print('Google made {} GPU available for this notebook.'.format(torch.cuda.device_count()))
  print('GPU type: {}'.format(torch.cuda.get_device_name()))
else:
  print('CPU must be used')
  device = torch.device("cpu")

Google made 2 GPU available for this notebook.
GPU type: Tesla V100-SXM2-32GB


In [5]:
rd = pd.read_csv("abstract_title_text_RD.csv", nrows = 4000)
rd = rd[rd['language'] == 'en']
rd = rd.dropna()
rd = rd.reset_index(drop=True)
rd['text_all'] = rd['title'].astype(str) + ',' + rd['abstract'].astype(str) + ',' + rd['text'].astype(str)
# metadata: review year ids
rd['year'] = pd.to_datetime(rd['date']).dt.year
year_dict = {k: k-rd['year'].min() for k in rd['year'].unique()}
rd['year_ids'] = rd['year'].replace(year_dict)
test_data = rd
# tranform data to lists
feature_text_target, feature_year_target = test_data['text_all'].to_list(), test_data['year_ids'].to_list()

In [6]:
# sample_data_df = pd.read_csv(path_to_input.format('sample_data.csv'))
sample_data_df = pd.read_csv('sample_data.csv', nrows = 4000)
possible_labels = sample_data_df['cpc_class'].unique()

numerical_encoding_dict = {}
for index, possible_label in enumerate(possible_labels):
    numerical_encoding_dict[possible_label] = index

sample_data_df['cpc_class_numerical'] = sample_data_df['cpc_class'].replace(numerical_encoding_dict)

sample_data_df['text'] = sample_data_df['patent_title'].astype(str) + ',' + sample_data_df['patent_abstract'].astype(str) + ',' + sample_data_df['summary_text'].astype(str)

# metadata: review year ids
sample_data_df['year'] = pd.to_datetime(sample_data_df['patent_date']).dt.year
year_dict = {k: k-sample_data_df['year'].min() for k in sample_data_df['year'].unique()}
sample_data_df['year_ids'] = sample_data_df['year'].replace(year_dict)

In [7]:
# train validation test hold out split
# test size
test_size = 0.2

# validation size
val_size = 0.2

# training size
train_size = 0.6

training_data, test_data = sklearn.model_selection.train_test_split(sample_data_df, test_size=test_size, random_state=parameter_dict['seed'])
training_data, validation_data = sklearn.model_selection.train_test_split(training_data, test_size=val_size/(1-test_size), random_state=parameter_dict['seed'])
# del all_data_df

In [8]:
# tranform data to lists
feature_text_train, feature_year_train, labels_train = training_data['text'].to_list(), training_data['year_ids'].to_list(), training_data['cpc_class_numerical'].to_list()
feature_text_validation, feature_year_validation, labels_validation = validation_data['text'].to_list(), validation_data['year_ids'].to_list(), validation_data['cpc_class_numerical'].to_list()
feature_text_test, feature_year_test, labels_test = test_data['text'].to_list(), test_data['year_ids'].to_list(), test_data['cpc_class_numerical'].to_list()

In [9]:
class BertClassificationMetadata(transformers.BertPreTrainedModel):
    def __init__(self, config, num_year_ids, metadata_embedding_size, hidden_layer_size, dropout_fine_tune):
        super().__init__(config)
        self.bert = transformers.BertModel(config)
        self.num_labels = config.num_labels
        self.config = config

        self.dropout = nn.Dropout(dropout_fine_tune)

        
        self.embedding_year = nn.Embedding(num_year_ids, metadata_embedding_size) #32, 128       
        self.layer_normalizer = nn.LayerNorm(768 + metadata_embedding_size) # 896
        self.fc1_hidden = nn.Linear(768 + metadata_embedding_size, hidden_layer_size)#896 , 2048
        self.fc2_hidden = nn.Linear(hidden_layer_size, hidden_layer_size)#2048 , 2048
        self.fc_classifier = nn.Linear(hidden_layer_size, config.num_labels)

        # Initialize weights and apply final processing

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        year_ids: Optional[torch.Tensor] = None,
    ) -> Union[Tuple[torch.Tensor], transformers.modeling_outputs.SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=True,
            return_dict=return_dict,
        )

        year_embedding = self.embedding_year(year_ids)

        pooled_bert_output = outputs[1]
        
        # print(pooled_bert_output.size(), year_embedding.size())
        
        # concat
        input_classification_head = torch.cat((pooled_bert_output, year_embedding), dim=1)#32, 1024

        # fc1
        input_fc1_hidden = self.dropout(self.layer_normalizer(input_classification_head))#32, 1024
        output_fc1_hidden = self.fc1_hidden(input_fc1_hidden)# 32, 768

        # fc2
        input_fc2_hidden = torch.relu(self.dropout(output_fc1_hidden))
        output_fc2_hidden = self.fc2_hidden(input_fc2_hidden) #32, 768

        # logit classifier
        input_fc_classifier = torch.relu(self.dropout(output_fc2_hidden))
        logits = self.fc_classifier(input_fc_classifier)

        # loss
        # loss_fct = nn.CrossEntropyLoss()
        # loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # loss
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        else:
            loss = None
            
        return transformers.modeling_outputs.SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [10]:
# Setup the pre-treained BERT Tokenizer
bert_tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Apply the tokenizer to the datasets
feature_text_train = bert_tokenizer(feature_text_train, padding='max_length', max_length=400, truncation =True, return_tensors='pt')
feature_text_validation = bert_tokenizer(feature_text_validation, padding='max_length', max_length=400, truncation =True, return_tensors='pt')
feature_text_test = bert_tokenizer(feature_text_test, padding='max_length', max_length=400, truncation =True, return_tensors='pt')

In [11]:
# tokenizer examples
# print('Example of the BERT tokenizer: \n \t {}'.format(bert_tokenizer.decode(feature_text_train.data['input_ids'][2])))

In [12]:
# data loader for transformers
class AttributeDataset(torch.utils.data.Dataset):
  def __init__(self, tokenized_text: torch.Tensor, year_ids, labels):
    self.tokenized_features = tokenized_text
    self.year_ids = torch.tensor(year_ids)
    self.labels = torch.tensor(labels)
  
  def __getitem__(self, idx):
    item = {key: val[idx] for key, val in self.tokenized_features.items()}
    item['year_ids'] = self.year_ids[idx]
    item['labels'] = self.labels[idx]
    return item
  
  def __len__(self):
    return len(self.labels)

train_torch = AttributeDataset(feature_text_train, feature_year_train, labels_train)
validation_torch = AttributeDataset(feature_text_validation, feature_year_validation, labels_validation)
test_torch = AttributeDataset(feature_text_test, feature_year_test, labels_test)

In [13]:
# Load the pretrained BERT model and push it to the GPU memory

#bert_classification_model = transformers.BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(numerical_encoding_dict))
bert_classification_model = BertClassificationMetadata.from_pretrained('bert-base-uncased', num_labels=len(numerical_encoding_dict), num_year_ids = max(year_dict.keys()), metadata_embedding_size=parameter_dict['metadata_embedding_size'], hidden_layer_size=parameter_dict['hidden_layer_size'], dropout_fine_tune=parameter_dict['dropout_finetune'])

# model to GPU
bert_classification_model.to(device)

# Define the data loader for batching, batch size 16 seems to work well
# (higher would cause memory overflow problems on the GPU)
train_data_loader = torch.utils.data.DataLoader(train_torch, batch_size=parameter_dict['batch_size'], shuffle=True)
validation_data_loader = torch.utils.data.DataLoader(validation_torch, batch_size=parameter_dict['batch_size'], shuffle=True)
number_of_batches = len(train_data_loader)
print(f'Number of Batches: {number_of_batches}')

# Setup the ADAM optimizer with generic parameters.
optimizer = torch.optim.AdamW(bert_classification_model.parameters(), lr=parameter_dict['learning_rate_AdamW'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertClassificationMetadata: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertClassificationMetadata from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertClassificationMetadata from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertClassificationMetadata were not initialized from the model checkpoint at bert-base-u

Number of Batches: 240


In [14]:
# Setup the pre-treained BERT Tokenizer
bert_tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Apply the tokenizer to the datasets
feature_text_target= bert_tokenizer(feature_text_target, padding='max_length', max_length=400, truncation =True, return_tensors='pt')

# data loader for transformers
class AttributeDataset(torch.utils.data.Dataset):
  def __init__(self, tokenized_text: torch.Tensor, year_ids):
    self.tokenized_features = tokenized_text
    self.year_ids = torch.tensor(year_ids)
  
  def __getitem__(self, idx):
    item = {key: val[idx] for key, val in self.tokenized_features.items()}
    item['year_ids'] = self.year_ids[idx]
    return item

  def __len__(self):
    return rd.shape[0]

target_torch = AttributeDataset(feature_text_target, feature_year_target)
target_dataloader = torch.utils.data.DataLoader(target_torch, batch_size=parameter_dict['batch_size'])


In [15]:
# milestones
def training_milestone(number_of_milestones, number_of_batches):
  batches_per_milestone = (number_of_batches-1)/number_of_milestones
  milestone_list = [int(x*batches_per_milestone) for x in range(1, number_of_milestones)]
  milestone_list.append(number_of_batches-1)
  return milestone_list

training_milestones = training_milestone(16, number_of_batches)

# validation
validation_milestones = training_milestone(4, number_of_batches)

In [16]:
def mmd_loss(hidden_states_source, hidden_states_target, sigma):
    # Compute the mean embeddings of the source and target domains
    mean_source = torch.mean(hidden_states_source, dim=0)
    mean_target = torch.mean(hidden_states_target, dim=0)

    # Compute the MMD between the source and target domains
    mmd = 0
    for i in range(hidden_states_source.shape[0]):
        for j in range(hidden_states_target.shape[0]):
            dist = torch.sum(torch.square(hidden_states_source[i] - hidden_states_target[j]))
            mmd += torch.exp(-dist / (2 * sigma ** 2))

    return mmd / (hidden_states_source.shape[0] * hidden_states_target.shape[0])


In [17]:
# start timer
start_time = time.time()
# initialize training
bert_classification_model.train()
# first loss to high values to not save model
sum_of_val_loss_min = 9999
# iterator for the number of validation runs
val_run_iter = 0

print(''.join([25*'-', ' Start Training ', 25*'-']))
print(''.join([25*'-', '----------------', 25*'-']))
alpha = 0.0001
sigma = 1.0
for epoch in range(parameter_dict['epochs']):
    for batch_iter, (batch_source, batch_target) in enumerate(zip(train_data_loader, target_dataloader)):
        # batch to GPU
        # text
        input_ids_source, attention_mask_source, year_ids_source, labels_source = batch_source['input_ids'].to(device), batch_source['attention_mask'].to(device), batch_source['year_ids'].to(device), batch_source['labels'].to(device)
        input_ids_target, attention_mask_target, year_ids_target = batch_target['input_ids'].to(device), batch_target['attention_mask'].to(device), batch_target['year_ids'].to(device)
        # after each updateing set optimzer gradient to zero
        optimizer.zero_grad()
        
        outputs_source = bert_classification_model(input_ids=input_ids_source, attention_mask=attention_mask_source, year_ids=year_ids_source, labels = labels_source)
        outputs_target = bert_classification_model(input_ids=input_ids_target, attention_mask=attention_mask_target, year_ids=year_ids_target)
        hidden_states_target = outputs_target.hidden_states[0]
        hidden_states_source = outputs_source.hidden_states[0]
        
        mmd_loss_value = mmd_loss(hidden_states_source, hidden_states_target, sigma)
        # Compute the distance between the source and target distributions
        target_dist = torch.mean(torch.exp(hidden_states_target), dim=0)
        source_dist = torch.mean(torch.exp(hidden_states_source), dim=0)
        
        """
        the code computes the source and target distributions by taking the mean of the exponential values of the hidden states. 
        The exponential function is used to convert the hidden states into a probability distribution over the vocabulary.
        The mean is computed along the batch dimension, resulting in a single probability distribution for each domain.
        """
        distance = torch.sum(torch.abs(source_dist - target_dist))
        """
        Finally, the code computes the distance between the source and target distributions using the L1 distance. 
        The L1 distance is the sum of the absolute differences between the corresponding elements of the two distributions. 
        This distance is used as a measure of how different the two domains are from each other. 
        The goal of domain adaptation is to minimize this distance, typically by adjusting the model's parameters to make it more robust to domain shift.
        """

        # total_loss = outputs_source[0] + alpha * distance
        
        """
        The MMD loss measures the difference between the source and target domains in terms of their embeddings in a high-dimensional space defined by the Gaussian kernel.
        The goal of the loss is to minimize this difference, which encourages the model to learn embeddings that are similar across domains.
        """
        
        # Compute the total loss as a linear combination of the CE and MMD losses
        loss = outputs_source[0] + alpha * mmd_loss_value
        # backpropergation
        loss.backward() 
        optimizer.step()

        # foward_pass_output['logits'] logits is the classifcation output
        # before the softmax is applied --> the prediction is always the highest
        # one.

        # logging
        training_log_dict = {
            'training_loss': torch.mean(loss)
            }

        
    # progress reporting 
    if batch_iter in training_milestones:
      # save memory by not calculating a gradient
      with torch.no_grad():
        print(f'Epoch: {epoch}\tBatch: {batch_iter}\tTrain-Loss: {torch.mean(loss)}')
    
    # Validation:
    # For each epoch I evaluate the model on a validation set twice. The model with 
    # the lowest (batch-wise sum) cross-entropy loss on the validation set is chosen.


    if batch_iter in validation_milestones:
      # set to evaluation (stop dropout)
      bert_classification_model.eval()
      print(''.join([25*'-', '----------------', 25*'-']))
      print(f'Validation Run {val_run_iter} (Epoch {epoch}):')

      # initaile selection cretarion.
      sum_of_val_loss = 0
      num_of_correct_predictions = 0
      # loop over validation loss
      for val_batch in validation_data_loader:
        # permit pytroch to save gradients for the validation data since no 
        # backpropergation is not needed (save memory)

        with torch.no_grad():
         
          # batch to GPU
          # text
          batch_text_inputids = val_batch['input_ids'].to(device)
          batch_text_attention_mask = val_batch['attention_mask'].to(device)

          # metadata
          batch_yearids = val_batch['year_ids'].to(device)

          #labels
          labels = val_batch['labels'].to(device)
    
          # forward pass
          forward_pass_output = bert_classification_model(input_ids=batch_text_inputids, attention_mask=batch_text_attention_mask, year_ids=batch_yearids, labels=labels)
          loss = forward_pass_output[0]

          # add batch specific loss
          sum_of_val_loss += loss

          # accuracy
          num_of_correct_predictions += torch.sum(forward_pass_output['logits'].argmax(axis=1) == labels)

      # log the loss and precision of the model

      mean_loss_val = sum_of_val_loss/len(validation_torch)*1024
      accuracy_val = num_of_correct_predictions/len(validation_torch)
      validation_log_dict = {
          'validation_loss': sum_of_val_loss/len(validation_torch)*1024,
          'validation_accuracy': num_of_correct_predictions/len(validation_torch)
        }

      # print validation results
      print(f'\t{val_run_iter}\n\t Val-Loss: \t{mean_loss_val}\n\t Val-Accu: \t{accuracy_val}')

      # update iter
      val_run_iter += 1
      # save the model if loss improved
      if (sum_of_val_loss<sum_of_val_loss_min):
        sum_of_val_loss_min = sum_of_val_loss
        torch.save(bert_classification_model, f'{model_name}.pt')
        print(''.join(['!'*25, ' New best Model ', '!'*25]))
      
      # print validation loss
      print(''.join([25*'-', '----------------', 25*'-']))
      print(f'Resume Training:')
      # set to training (activate dropout)
      bert_classification_model.train()

# end training           
bert_classification_model.eval() 
print('The Training took: ',time.time()-start_time, 'Seconds')
del bert_classification_model
    

------------------------- Start Training -------------------------
------------------------------------------------------------------
Epoch: 0	Batch: 239	Train-Loss: 2.953341007232666
------------------------------------------------------------------
Validation Run 0 (Epoch 0):
	0
	 Val-Loss: 	309.0259094238281
	 Val-Accu: 	0.3449999988079071
!!!!!!!!!!!!!!!!!!!!!!!!! New best Model !!!!!!!!!!!!!!!!!!!!!!!!!
------------------------------------------------------------------
Resume Training:
Epoch: 1	Batch: 239	Train-Loss: 1.5150697231292725
------------------------------------------------------------------
Validation Run 1 (Epoch 1):
	1
	 Val-Loss: 	232.5215606689453
	 Val-Accu: 	0.5149999856948853
!!!!!!!!!!!!!!!!!!!!!!!!! New best Model !!!!!!!!!!!!!!!!!!!!!!!!!
------------------------------------------------------------------
Resume Training:
Epoch: 2	Batch: 239	Train-Loss: 2.087878465652466
------------------------------------------------------------------
Validation Run 2 (Epoch 

In [18]:
# loading the fine tuned model chosen on the validation set
classification_model = torch.load((f'{model_name}.pt'), map_location=torch.device(device))

In [26]:
# ignore dropout for testing
classification_model.eval()

# timer
start_time = time.time()

# load test data
test_data_loader = torch.utils.data.DataLoader(test_torch, batch_size=parameter_dict['batch_size'])

# numpy arrays to store predictions and true labels.
predictions = np.array([])
true_labels = np.array([])

# train the model
for test_batch in test_data_loader:

    # I do not need gradient
    with torch.no_grad():

        # batch to GPU
        # text
        batch_text_inputids = test_batch['input_ids'].to(device)
        batch_text_attention_mask = test_batch['attention_mask'].to(device)

        # metadata
        batch_yearids = test_batch['year_ids'].to(device)

        #labels
        labels = test_batch['labels'].to(device)

        # forward pass
        foward_pass_output = classification_model(input_ids=batch_text_inputids, attention_mask=batch_text_attention_mask, year_ids=batch_yearids, labels=labels)
        
        # foward_pass_output['logits'] logits is the classifcation output
        # before the softmax is applied --> the prediction is always the highest
        # one.
        predictions = np.append(predictions, foward_pass_output['logits'].argmax(axis=1).cpu().numpy())

        # since I shuffle the batche
        true_labels = np.append(true_labels, labels.cpu().numpy())

print('Out-of-sample predictions took: ',time.time()-start_time, 'Seconds')

Out-of-sample predictions took:  5.939906120300293 Seconds


In [27]:
# model evaluations
import tables
Path((f'{model_name}')).mkdir(parents=True, exist_ok=True)

# tests
test_statistic = dict()
test_statistic['accuracy'] = sklearn.metrics.accuracy_score(true_labels, predictions)
test_statistic['recall'] = sklearn.metrics.recall_score(true_labels, predictions, average='macro')
test_statistic['precison'] = sklearn.metrics.precision_score(true_labels, predictions, average='macro')
test_statistic['f1_score'] = sklearn.metrics.f1_score(true_labels, predictions, average='macro')

# # to df and safe
scores_df = pd.DataFrame.from_dict(test_statistic, orient='index').reset_index().rename(columns={'index': 'measure', 0: 'score'})
scores_df.to_csv((f'{model_name}/measures_{model_name}_test.csv'), index=False)
print(f'Evaluation Measures Test Set:\n{scores_df.to_markdown()}\n')

# # get the confusion matrix
# confusion_matrix = sklearn.metrics.confusion_matrix(true_labels, predictions)
# confusion_matrix_plot = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix.astype('int'), display_labels=numerical_encoding_dict.keys())
# confusion_matrix_plot.plot(values_format='.0f', xticks_rotation='vertical')
# print('Confusion Matrix Test Set:')
# plt.tight_layout()
# plt.savefig((f'{model_name}/confusion_matirx_{model_name}_test.png'), bbox_inches='tight', dpi=1080)

Evaluation Measures Test Set:
|    | measure   |    score |
|---:|:----------|---------:|
|  0 | accuracy  | 0.61875  |
|  1 | recall    | 0.250914 |
|  2 | precison  | 0.206538 |
|  3 | f1_score  | 0.217989 |



/home/ma/ma_ma/ma_ssureshb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
# predictions

In [22]:
with open('predictions.txt', 'w') as file:
    for item in predictions:
        file.write("%s\n" % item)

In [23]:
predictions_list = list(map(int, predictions))


In [24]:
matched_keys = []

for prediction in predictions_list:
    for key, value in numerical_encoding_dict.items():
        if value == prediction:
            matched_keys.append(key)
            break

In [25]:
rd['Predicted_Label'] = matched_keys

ValueError: Length of values (800) does not match length of index (3920)

In [ ]:
rd_predicted = rd[['rdid', 'title', 'abstract', 'date', 'year', 'Predicted_Label']]

In [ ]:
rd_predicted.to_csv("rd_predicted.csv")